In [3]:
import pandas as pd 
from dotenv import load_dotenv
import os 

load_dotenv()

True

In [7]:
movies_df = pd.read_csv(r"C:\Users\bhavi\OneDrive\Desktop\langhcain_learning\RAG\IMDB top 1000.csv")
movies_df[["Title","Metascore","Description","Cast"]].head(2)

,Title,Metascore,Description,Cast
0,1. The Shawshank Redemption (1994),80.0,Two imprisoned men bond over a number of years...,"Director: Frank Darabont | Stars: Tim Robbins,..."
1,2. The Godfather (1972),100.0,The aging patriarch of an organized crime dyna...,Director: Francis Ford Coppola | Stars: Marlon...


In [12]:
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_groq import ChatGroq
from langchain_chroma import Chroma

In [ ]:
embeddings = HuggingFaceEmbeddings(model_name = "all-MiniLM-L6-v2")